In [81]:
from sklearn import datasets, metrics, model_selection, tree, ensemble
from numpy import linalg

In [61]:
def write_answer(val, name):
    with open(name, "w") as fp:
        fp.write(val)

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [62]:
X, Y = datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.25, shuffle=False)

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [75]:
def classify_1nn(x_train, y_train, vector):
    distances = [(linalg.norm(x - vector), y_train[i]) for i, x in enumerate(x_train)]
    distances = sorted(distances)
    return distances[0][1]

In [70]:
def error_share(xtrain, ytrain, xtest, ytest):
    return sum((1 if classify_1nn(xtrain, ytrain, xtest[i]) != ytest[i] else 0) for i in range(len(xtest))) / len(xtest)

In [76]:
classify_1nn(X_train, y_train, X_test[0])

3

In [77]:
error_share(X_train, y_train, X_train, y_train)

0.0

In [78]:
error_share = error_share(X_train, y_train, X_test, y_test)
print(f'Errors share: {error_share}')

Errors share: 0.03777777777777778


In [79]:
write_answer(str(error_share), '3answer1.txt')

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [85]:
classifier = ensemble.RandomForestClassifier(n_estimators=1000)
classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print(f'Score: {score}')
error_share_2 = 1 - score
print(f'Score: {error_share_2}')

Score: 0.9244444444444444
Score: 0.0755555555555556


In [86]:
write_answer(str(error_share_2), '3answer2.txt')